# Import required libraries

In [1]:
# pip install seaborn
# pip install xgboost
# pip install keras
# pip install sklearn
# pip install pandas
# pip install numpy

In [1]:
import pandas as pd

In [2]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

Using TensorFlow backend.


# Import the data

In [3]:
# help(pd.read_excel)

In [4]:
# df_x = pd.read_excel('Research data sample.xlsx', sheet_name = 'Sheet1')

In [5]:
df_x = pd.read_excel('Research data.xlsx', sheet_name = 'Sheet2')

In [6]:
df_x.head(2)

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,...,Column12,Column13,Column14,Column15,Column16,Column17,Column18,Column19,Column20,Column21
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [7]:
df_x.columns

Index(['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6',
       'Column7', 'Column8', 'Column9', 'Column10', 'Column11', 'Column12',
       'Column13', 'Column14', 'Column15', 'Column16', 'Column17', 'Column18',
       'Column19', 'Column20', 'Column21'],
      dtype='object')

In [8]:
# df_y = pd.read_excel('Research data sample.xlsx', sheet_name = 'Sheet2')

In [9]:
df_y = pd.read_excel('Research data.xlsx', sheet_name = 'Sheet3')

In [10]:
df_y.head(2)

,Column1
0,66.193
1,66.193


In [11]:
df_y = df_y.rename(columns={'Column1': 'Position'})

In [12]:
df_y.head(2)

,Position
0,66.193
1,66.193


In [13]:
df_all = df_x
df_all['Position'] = df_y['Position']

In [14]:
# df_all.to_csv('all-combined-data.csv', index = False)

In [15]:
len(df_x)

729633

In [16]:
train_x = df_x[0:500000]
test_x = df_x[600000:]
train_y = df_y[0:500000]
test_y = df_y[600000:]

In [17]:
val_x = df_x[500000:600000]
val_y = df_y[500000:600000]

In [18]:
# df_all.head(2)

In [19]:
# from sklearn.model_selection import train_test_split

# train_x, test_x, train_y, test_y = train_test_split(df_x, df_y, test_size=0.2, random_state=42)

In [20]:
train_x.head(2)

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,...,Column13,Column14,Column15,Column16,Column17,Column18,Column19,Column20,Column21,Position
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,66.193
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,66.193


In [21]:
len(test_x)

129633

In [22]:
train_x.shape[1]

22

# Training a Simple Logistic Regression Model

#### Followed this blogpost: https://aigeekprogrammer.com/binary-classification-using-logistic-regression-and-keras/

In [23]:
LR_model = Sequential()

# The Input Layer :
LR_model.add(Dense(train_x.shape[1], kernel_initializer='normal',input_dim = train_y.shape[1], activation='softmax'))

# Compile the network :
LR_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
LR_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 22)                44        
Total params: 44
Trainable params: 44
Non-trainable params: 0
_________________________________________________________________


#### Create a folder call "lr_models" to save your model weights

In [24]:
checkpoint_name = 'lr_models_decoding/Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [25]:
callbacks_list

In [26]:
train = train_x.to_numpy()

In [27]:
train.shape

(500000, 22)

In [28]:
target = train_y.to_numpy()

In [29]:
target.shape

(500000, 1)

In [42]:
# ?NN_model.fit

In [43]:
val_y.to_numpy().shape

(100000, 1)

In [30]:
LR_model.fit(target, train, epochs=50, batch_size=64, validation_data = (val_y.to_numpy(), val_x.to_numpy()), callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Train on 500000 samples, validate on 100000 samples
Epoch 1/50
500000/500000 [==============================] - 20s 40us/step - loss: 8.2197 - mean_absolute_error: 8.2197 - val_loss: 7.9675 - val_mean_absolute_error: 7.9675

Epoch 00001: val_loss improved from inf to 7.96749, saving model to lr_models_decoding/Weights-001--7.96749.hdf5
Epoch 2/50
500000/500000 [==============================] - 18s 37us/step - loss: 8.2184 - mean_absolute_error: 8.2184 - val_loss: 7.9675 - val_mean_absolute_error: 7.9675

Epoch 00002: val_loss improved from 7.96749 to 7.96749, saving model to lr_models_decoding/Weights-002--7.96749.hdf5
Epoch 3/50
500000/500000 [==============================] - 20s 40us/step - loss: 8.2184 - mean_absolute_error: 8.2184 - val_loss: 7.9675 - val_mean_absolute_error: 7.9675

Epoch 00003: val_loss improved from 7.96749 to 7.96749, saving model to lr_models_decoding/Weights-003--7.96749.hdf5
Epoch 4/50
500000/500000 [========

500000/500000 [==============================] - 25s 50us/step - loss: 8.2184 - mean_absolute_error: 8.2184 - val_loss: 7.9675 - val_mean_absolute_error: 7.9675

Epoch 00029: val_loss improved from 7.96748 to 7.96748, saving model to lr_models_decoding/Weights-029--7.96748.hdf5
Epoch 30/50
500000/500000 [==============================] - 26s 52us/step - loss: 8.2184 - mean_absolute_error: 8.2184 - val_loss: 7.9675 - val_mean_absolute_error: 7.9675

Epoch 00030: val_loss improved from 7.96748 to 7.96748, saving model to lr_models_decoding/Weights-030--7.96748.hdf5
Epoch 31/50
500000/500000 [==============================] - 32s 64us/step - loss: 8.2184 - mean_absolute_error: 8.2184 - val_loss: 7.9675 - val_mean_absolute_error: 7.9675

Epoch 00031: val_loss improved from 7.96748 to 7.96748, saving model to lr_models_decoding/Weights-031--7.96748.hdf5
Epoch 32/50
500000/500000 [==============================] - 22s 43us/step - loss: 8.2184 - mean_absolute_error: 8.2184 - val_loss: 7.9675 

In [31]:
wights_file = 'lr_models_decoding/Weights-050--7.96748.hdf5' # choose the best checkpoint 
LR_model.load_weights(wights_file) # load it
LR_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [32]:
test = test_x.to_numpy()

In [42]:
actual = test_y.to_numpy()

In [43]:
actual[1]

array([215.35])

In [44]:
predictions = LR_model.predict(actual)

In [46]:
predictions[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1.], dtype=float32)

### We will judge our model on Mean Absolute Error Metric

In [47]:
MAE = mean_absolute_error(test , predictions)
print('Deep Neural Network validation MAE = ', MAE)

Deep Neural Network validation MAE =  7.469172929561964
